In [12]:
# Import modules
import pandas as pd
import os
import requests
import json
from pandas import json_normalize
import numpy as np

In [13]:
# google places api info
gp_secret = os.environ["gp_api"]

In [14]:
gp_resto = pd.read_csv('google_places_poi.csv')
gp_school = pd.read_csv('google_places_school.csv')
gp_park = pd.read_csv('google_places_park.csv')
gp_transit = pd.read_csv('google_places_transit.csv')
gp_grocery = pd.read_csv('google_places_supermarket.csv')


In [15]:
# creating one large dataframe
gp_pois =  pd.concat([gp_resto,gp_school,gp_park,gp_transit,gp_grocery])

In [16]:
gp_pois.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26272 entries, 0 to 2448
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   business_status                  26108 non-null  object 
 1   icon                             26272 non-null  object 
 2   icon_background_color            26272 non-null  object 
 3   icon_mask_base_uri               26272 non-null  object 
 4   name                             26272 non-null  object 
 5   photos                           17188 non-null  object 
 6   place_id                         26272 non-null  object 
 7   price_level                      5248 non-null   float64
 8   rating                           19053 non-null  float64
 9   reference                        26272 non-null  object 
 10  scope                            26272 non-null  object 
 11  types                            26272 non-null  object 
 12  user_ratings_total 

## We need to run an additional API query on every location in order to get proper address info

In [ ]:

def gp_locations (gp_id):
    
    # construct url
    url= 'https://maps.googleapis.com/maps/api/place/details/json?place_id=' + gp_id + '&key=' + gp_secret
    
    # create dictionary for headers
    headers = {}
    #payload = {}
        
    # perform get request
    response = requests.get(url, headers=headers)

    # process request into usable JSON file
    gp_response = json.loads(response.text)
    gp_data.append(gp_response)
    
    return


In [ ]:
gp_data=[]
def gp_api_loop():

    num_rows = gp_pois.shape[0]
    row_count = 0


    while row_count < num_rows:
        gp_locations((gp_pois.iloc[row_count,6]))
        row_count +=1

In [ ]:
gp_api_loop()

In [ ]:
# saving results as json files for convenience
with open('gp_raw.json', 'w') as f:
    json.dump(gp_data, f)

In [ ]:
with open('gp_raw.json') as f:
    gp_data = json.load(f)

In [ ]:
# normalizing the json file
gp_raw_df = pd.json_normalize(gp_data)

In [510]:
# investigating results
gp_raw_df.head()

,html_attributions,status,result.address_components,result.adr_address,result.business_status,result.current_opening_hours.open_now,result.current_opening_hours.periods,result.current_opening_hours.weekday_text,result.delivery,result.dine_in,...,result.utc_offset,result.vicinity,result.website,result.wheelchair_accessible_entrance,result.serves_breakfast,result.serves_lunch,result.serves_vegetarian_food,result.curbside_pickup,result.permanently_closed,result.current_opening_hours.special_days
0,[],OK,"[{'long_name': '151', 'short_name': '151', 'ty...","<span class=""street-address"">151 W Sandford Bl...",OPERATIONAL,False,"[{'close': {'date': '2022-11-29', 'day': 2, 't...","[Monday: Closed, Tuesday: 4:00 – 11:00 PM, Wed...",False,True,...,-300,"151 West Sandford Boulevard, Mount Vernon",http://www.riperestaurant.com/,False,NaN,NaN,NaN,NaN,NaN,NaN
1,[],OK,"[{'long_name': '724', 'short_name': '724', 'ty...","<span class=""street-address"">724 Nereid Ave</s...",OPERATIONAL,False,"[{'close': {'date': '2022-12-04', 'day': 0, 't...","[Monday: 11:00 AM – 10:30 PM, Tuesday: 11:00 A...",True,NaN,...,-300,"724 Nereid Avenue, The Bronx",NaN,NaN,False,True,True,NaN,NaN,NaN
2,[],OK,"[{'long_name': '980', 'short_name': '980', 'ty...","<span class=""street-address"">980 E 233rd St</s...",OPERATIONAL,False,"[{'close': {'date': '2022-12-04', 'day': 0, 't...","[Monday: 6:00 AM – 9:00 PM, Tuesday: 6:00 AM –...",True,True,...,-300,"980 East 233rd Street, The Bronx",https://locations.dunkindonuts.com/en/ny/bronx...,True,True,True,NaN,False,NaN,NaN
3,[],OK,"[{'long_name': '980', 'short_name': '980', 'ty...","<span class=""street-address"">980 E 233rd St</s...",OPERATIONAL,False,"[{'close': {'date': '2022-12-04', 'day': 0, 't...","[Monday: 10:00 AM – 10:00 PM, Tuesday: 10:00 A...",True,True,...,-300,"980 East 233rd Street, The Bronx",https://restaurants.subway.com/united-states/n...,True,True,True,True,NaN,NaN,NaN
4,[],OK,"[{'long_name': '1201', 'short_name': '1201', '...","<span class=""street-address"">1201 E 233rd St</...",OPERATIONAL,False,"[{'close': {'date': '2022-12-04', 'day': 0, 't...","[Monday: 10:00 AM – 12:00 AM, Tuesday: 10:00 A...",True,True,...,-300,"1201 East 233rd Street, The Bronx",https://www.popeyes.com/store-locator/store/re...,True,NaN,True,NaN,NaN,NaN,NaN


In [511]:
gp_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26272 entries, 0 to 26271
Data columns (total 56 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   html_attributions                          26272 non-null  object 
 1   status                                     26272 non-null  object 
 2   result.address_components                  26272 non-null  object 
 3   result.adr_address                         26272 non-null  object 
 4   result.business_status                     26108 non-null  object 
 5   result.current_opening_hours.open_now      13839 non-null  object 
 6   result.current_opening_hours.periods       13839 non-null  object 
 7   result.current_opening_hours.weekday_text  13839 non-null  object 
 8   result.delivery                            6985 non-null   object 
 9   result.dine_in                             5140 non-null   object 
 10  result.editorial_summa

In [ ]:
# exporting as csv
gp_pois.to_csv('gp_pois_raw.csv', index=False)

# Making Dataframe Containing Only Addresses

In [ ]:
# making dataframe of only addresses
addresses = pd.DataFrame(gp_raw_df['result.formatted_address'])

In [512]:
addresses.head()

,result.formatted_address,Borough,Zip,Neighbourhood
0,"151 W Sandford Blvd, Mt Vernon, NY 10550, USA",nan,nan,nan
1,"724 Nereid Ave, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
2,"980 E 233rd St, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
3,"980 E 233rd St, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
4,"1201 E 233rd St, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx


In [2]:
# importing csvs of nyc neighbourhood info
nyc_nb = pd.read_csv('nyc_neighbs.csv',encoding='cp1252')
nyc_bo = pd.read_csv('nyc_boroughs.csv')

In [3]:
nyc_nb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 42 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Port Richmond                                          3 non-null      float64
 1   South Beach and Tottenville                            3 non-null      float64
 2   Stapleton and St. George                               3 non-null      float64
 3   Mid-Island (Willowbrook)                               1 non-null      float64
 4   Inwood and Washington Heights                          5 non-null      float64
 5   Central Harlem                                         5 non-null      float64
 6   East Harlem                                            2 non-null      float64
 7   Upper West Side                                        3 non-null      float64
 8   Upper East Side                                       

In [4]:
nyc_nb.head()

,Port Richmond,South Beach and Tottenville,Stapleton and St. George,Mid-Island (Willowbrook),Inwood and Washington Heights,Central Harlem,East Harlem,Upper West Side,Upper East Side,Chelsea and Clinton,...,Northwest Brooklyn,Central Brooklyn (Bedford Stuyvesant – Crown Heights),East New York and New Lots,Sunset Park,Southwest Brooklyn,Borough Park,Canarsie and Flatlands,Southern Brooklyn,Flatbush,Bushwick and Williamsburg
0,10302.0,10306.0,10301.0,10314.0,10031.0,10026.0,10029.0,10023.0,10021.0,10001.0,...,11201.0,11212.0,11207.0,11220.0,11209.0,11204.0,11234.0,11223.0,11203.0,11206.0
1,10303.0,10307.0,10304.0,NaN,10032.0,10027.0,10035.0,10024.0,10028.0,10011.0,...,11205.0,11213.0,11208.0,11232.0,11214.0,11218.0,11236.0,11224.0,11210.0,11221.0
2,10310.0,10308.0,10305.0,NaN,10033.0,10030.0,NaN,10025.0,10044.0,10018.0,...,11215.0,11216.0,NaN,NaN,11228.0,11219.0,11239.0,11229.0,11225.0,11237.0
3,NaN,NaN,NaN,NaN,10034.0,10037.0,NaN,NaN,10065.0,10019.0,...,11217.0,11233.0,NaN,NaN,NaN,11230.0,NaN,11235.0,11226.0,NaN
4,NaN,NaN,NaN,NaN,10040.0,10039.0,NaN,NaN,10075.0,10020.0,...,11231.0,11238.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# converting numbers to strings so we can search using them as our substring value
nyc_nb = nyc_nb.astype(str)
# removing trailing '.0'
nyc_nb = nyc_nb.stack().str.replace(r'\.0','').unstack()

C:\Users\annie\AppData\Local\Temp\ipykernel_51116\2168325353.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  nyc_nb = nyc_nb.stack().str.replace(r'\.0','').unstack()


In [6]:
nyc_nb

,Port Richmond,South Beach and Tottenville,Stapleton and St. George,Mid-Island (Willowbrook),Inwood and Washington Heights,Central Harlem,East Harlem,Upper West Side,Upper East Side,Chelsea and Clinton,...,Northwest Brooklyn,Central Brooklyn (Bedford Stuyvesant – Crown Heights),East New York and New Lots,Sunset Park,Southwest Brooklyn,Borough Park,Canarsie and Flatlands,Southern Brooklyn,Flatbush,Bushwick and Williamsburg
0,10302,10306,10301,10314,10031,10026,10029,10023,10021,10001,...,11201,11212,11207,11220,11209,11204,11234,11223,11203,11206
1,10303,10307,10304,nan,10032,10027,10035,10024,10028,10011,...,11205,11213,11208,11232,11214,11218,11236,11224,11210,11221
2,10310,10308,10305,nan,10033,10030,nan,10025,10044,10018,...,11215,11216,nan,nan,11228,11219,11239,11229,11225,11237
3,nan,nan,nan,nan,10034,10037,nan,nan,10065,10019,...,11217,11233,nan,nan,nan,11230,nan,11235,11226,nan
4,nan,nan,nan,nan,10040,10039,nan,nan,10075,10020,...,11231,11238,nan,nan,nan,nan,nan,nan,nan,nan
5,nan,nan,nan,nan,nan,nan,nan,nan,10128,10036,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [7]:
# converting numbers to strings so we can search using them as our substring value
nyc_bo = nyc_bo.astype(str)
# removing trailing '.0'
nyc_bo = nyc_bo.stack().str.replace(r'\.0','').unstack()

C:\Users\annie\AppData\Local\Temp\ipykernel_51116\178317327.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  nyc_bo = nyc_bo.stack().str.replace(r'\.0','').unstack()


In [8]:
nyc_bo.head()

,Manhattan,Staten Island,Bronx,Queens,Brooklyn
0,10001,10301,10451,11004,11201
1,10002,10302,10452,11005,11202
2,10003,10303,10453,11006,11203
3,10004,10304,10454,11007,11204
4,10005,10305,10455,11008,11205


In [ ]:
addresses.head()

In [ ]:
def assign_borough(borough):
    """ Finds borough name for address dataframe.
    
    Loops through the nyc_bo (nyc boroughs) dataframe to see if one of its zipcodes 
    is within the address string of our addresses dataframe. Then fills in borough name & cleanly formatted zipcode.
    
    Params:
        borough: desired borough (column from nyc_bo dataframe)
        
    Returns:
        fills 'Borough' value with column name from nyc_bo
        fills 'Zip' value with zipcode from nyc_bo table
    
    """
    num_rows = nyc_bo.shape[0]
    row_count = 0

    while row_count < num_rows:
        addresses.loc[addresses['result.formatted_address'].str.contains(nyc_bo[borough][row_count]), 'Borough'] = borough
        addresses.loc[addresses['result.formatted_address'].str.contains(nyc_bo[borough][row_count]), 'Zip'] = nyc_bo[borough][row_count]
        row_count +=1

In [519]:
nyc_bo.columns.tolist()

['Manhattan', 'Staten Island', 'Bronx', 'Queens', 'Brooklyn']

In [ ]:
assign_borough('Brooklyn')

In [ ]:
addresses.info()

In [ ]:
addresses.head()

In [ ]:
nyc_nb.columns.tolist()

In [ ]:
def assign_neighbourhood(neighbourhood):
    """ Finds neighbourhood name for address dataframe.
    
    Loops through the nyc_nb (nyc neighbourhood) dataframe to see if one of its zipcodes 
    is within the address string of our addresses dataframe. Then fills in borough name & cleanly formatted zipcode.
    
    Params:
        borough: desired borough (column from nyc_bo dataframe)
        
    Returns:
        fills 'neighbourhood' value with column name from nyc_nb
        fills 'Zip' value with zipcode from nyc_nb table
    
    """
    num_rows = nyc_nb.shape[0]
    row_count = 0

    while row_count < num_rows:
        addresses.loc[addresses['result.formatted_address'].str.contains(nyc_nb[neighbourhood][row_count]), 'Neighbourhood'] = neighbourhood
        addresses.loc[addresses['result.formatted_address'].str.contains(nyc_nb[neighbourhood][row_count]), 'Zip'] = nyc_nb[neighbourhood][row_count]
        row_count +=1

In [ ]:
neighbourhoods = nyc_nb.columns.tolist()

In [ ]:
nyc_nb.columns.tolist()

In [ ]:
# i'm sure there's a much more elegant way to do this
"""
a loop that iterates through a list of neighbourhood names, 
and inserts them into the phrasing needed for the assign_neighbourhood function
"""
for i in range(len(neighbourhoods)):
    print('assign_neighbourhood(\'' + neighbourhoods[i] + '\')')

In [ ]:
assign_neighbourhood('Port Richmond')
assign_neighbourhood('South Beach and Tottenville')
assign_neighbourhood('Stapleton and St. George')
assign_neighbourhood('Mid-Island (Willowbrook)')
assign_neighbourhood('Inwood and Washington Heights')
assign_neighbourhood('Central Harlem')
assign_neighbourhood('East Harlem')
assign_neighbourhood('Upper West Side')
assign_neighbourhood('Upper East Side')
assign_neighbourhood('Chelsea and Clinton')
assign_neighbourhood('Gramercy Park and Murray Hill')
assign_neighbourhood('Greenwich Village and Soho')
assign_neighbourhood('Union Square and Lower East Side')
assign_neighbourhood('Lower Manhattan')
assign_neighbourhood('Kingsbridge and Riverdale')
assign_neighbourhood('Northeast Bronx')
assign_neighbourhood('Crotona and Tremont')
assign_neighbourhood('Bronx Park and Fordham')
assign_neighbourhood('Pelham and Throgs Neck')
assign_neighbourhood('HighBridge and Morrisania')
assign_neighbourhood('Hunts Point and Mott Haven')
assign_neighbourhood('Long Island City and Astoria')
assign_neighbourhood('West Queens')
assign_neighbourhood('North Queens')
assign_neighbourhood('Northeast Queens')
assign_neighbourhood('West Central Queens')
assign_neighbourhood('Central Queens')
assign_neighbourhood('Southwest Queens')
assign_neighbourhood('Jamaica')
assign_neighbourhood('Southeast Queens')
assign_neighbourhood('Rockaways')
assign_neighbourhood('Greenpoint')
assign_neighbourhood('Northwest Brooklyn ')
assign_neighbourhood('Central Brooklyn (Bedford Stuyvesant – Crown Heights)')
assign_neighbourhood('East New York and New Lots')
assign_neighbourhood('Sunset Park')
assign_neighbourhood('Southwest Brooklyn')
assign_neighbourhood('Borough Park')
assign_neighbourhood('Canarsie and Flatlands')
assign_neighbourhood('Southern Brooklyn')
assign_neighbourhood('Flatbush')
assign_neighbourhood('Bushwick and Williamsburg')

In [520]:
addresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26272 entries, 0 to 26271
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   result.formatted_address  26272 non-null  object
 1   Borough                   26272 non-null  object
 2   Zip                       26272 non-null  object
 3   Neighbourhood             26272 non-null  object
dtypes: object(4)
memory usage: 821.1+ KB


In [521]:
addresses

,result.formatted_address,Borough,Zip,Neighbourhood
0,"151 W Sandford Blvd, Mt Vernon, NY 10550, USA",nan,nan,nan
1,"724 Nereid Ave, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
2,"980 E 233rd St, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
3,"980 E 233rd St, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
4,"1201 E 233rd St, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
...,...,...,...,...
26267,"686 Main St, New York, NY 10044, USA",Manhattan,10044,Upper East Side
26268,"375 Tompkins Ave., Staten Island, NY 10305, USA",Staten Island,10305,Stapleton and St. George
26269,"515-517 Targee St, Staten Island, NY 10304, USA",Staten Island,10304,Stapleton and St. George
26270,"523 Targee St, Staten Island, NY 10304, USA",Staten Island,10304,Stapleton and St. George


# Merging Dataframes

In [522]:
gp_pois.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26272 entries, 0 to 26271
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   business_status                  26108 non-null  object 
 1   icon                             26272 non-null  object 
 2   icon_background_color            26272 non-null  object 
 3   icon_mask_base_uri               26272 non-null  object 
 4   name                             26272 non-null  object 
 5   photos                           17188 non-null  object 
 6   place_id                         26272 non-null  object 
 7   price_level                      5248 non-null   float64
 8   rating                           19053 non-null  float64
 9   reference                        26272 non-null  object 
 10  scope                            26272 non-null  object 
 11  types                            26272 non-null  object 
 12  user_ratings_total

In [526]:
gp_pois.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26272 entries, 0 to 26271
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   business_status                  26108 non-null  object 
 1   icon                             26272 non-null  object 
 2   icon_background_color            26272 non-null  object 
 3   icon_mask_base_uri               26272 non-null  object 
 4   name                             26272 non-null  object 
 5   photos                           17188 non-null  object 
 6   place_id                         26272 non-null  object 
 7   price_level                      5248 non-null   float64
 8   rating                           19053 non-null  float64
 9   reference                        26272 non-null  object 
 10  scope                            26272 non-null  object 
 11  types                            26272 non-null  object 
 12  user_ratings_total

In [524]:
addresses.shape[0]

26272

In [528]:
addresses.head()

,result.formatted_address,Borough,Zip,Neighbourhood
0,"151 W Sandford Blvd, Mt Vernon, NY 10550, USA",nan,nan,nan
1,"724 Nereid Ave, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
2,"980 E 233rd St, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
3,"980 E 233rd St, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
4,"1201 E 233rd St, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx


In [529]:
gp_pois.head()

,business_status,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,price_level,rating,reference,...,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,permanently_closed,req_cat
0,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Ripe Kitchen & Bar,"[{'height': 2471, 'html_attributions': ['<a hr...",ChIJ9aOdYiyNwokRS49czrNTQo4,2.0,4.3,ChIJ9aOdYiyNwokRS49czrNTQo4,...,-73.838855,40.899498,-73.837452,40.896800,-73.840150,True,"V5X6+7F Mount Vernon, NY, USA",87G8V5X6+7F,NaN,restaurant
1,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,New China Garden,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJcVDAYNfywokRPUaKrm0UBpg,1.0,3.9,ChIJcVDAYNfywokRPUaKrm0UBpg,...,-73.853364,40.899306,-73.852100,40.896608,-73.854798,True,"V4XW+5M The Bronx, NY, USA",87G8V4XW+5M,NaN,restaurant
2,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Dunkin',"[{'height': 3000, 'html_attributions': ['<a hr...",ChIJ--58aynzwokRqSvaRDKwX1M,1.0,3.8,ChIJ--58aynzwokRqSvaRDKwX1M,...,-73.849089,40.891936,-73.847664,40.889239,-73.850362,True,"V5R2+59 New York, NY, USA",87G8V5R2+59,NaN,restaurant
3,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Subway,"[{'height': 766, 'html_attributions': ['<a hre...",ChIJAy-ibCnzwokRdmkKGd1Yzhw,1.0,3.6,ChIJAy-ibCnzwokRdmkKGd1Yzhw,...,-73.849152,40.891828,-73.847659,40.889130,-73.850357,True,"V5R2+58 New York, NY, USA",87G8V5R2+58,NaN,restaurant
4,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Popeyes Louisiana Kitchen,"[{'height': 1192, 'html_attributions': ['<a hr...",ChIJXVacFSvzwokRa3VpQDuwRDE,1.0,3.8,ChIJXVacFSvzwokRa3VpQDuwRDE,...,-73.843383,40.890762,-73.842012,40.888064,-73.844710,True,"V5Q4+QJ New York, NY, USA",87G8V5Q4+QJ,NaN,restaurant


In [525]:
gp_pois.shape[0]

26272

In [ ]:
# checking number of unique values pre-merge
print(len(gp_pois['place_id'].unique()))

In [ ]:
addresses = addresses.astype(str)

In [530]:
addresses.head()

,result.formatted_address,Borough,Zip,Neighbourhood
0,"151 W Sandford Blvd, Mt Vernon, NY 10550, USA",nan,nan,nan
1,"724 Nereid Ave, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
2,"980 E 233rd St, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
3,"980 E 233rd St, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
4,"1201 E 233rd St, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx


In [531]:
addresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26272 entries, 0 to 26271
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   result.formatted_address  26272 non-null  object
 1   Borough                   26272 non-null  object
 2   Zip                       26272 non-null  object
 3   Neighbourhood             26272 non-null  object
dtypes: object(4)
memory usage: 821.1+ KB


In [ ]:
# resetting index on both dataframes in order to merge
addresses.reset_index(drop=True, inplace=True)
gp_pois.reset_index(drop=True, inplace=True)

In [ ]:
addresses

In [ ]:
gp_pois

In [ ]:
gp_pois_merged = pd.concat([gp_pois, addresses], join='outer', ignore_index=False)

In [ ]:
gp_pois_merged = gp_pois.merge(addresses, how='outer', left_index=True, right_index=True)

In [534]:
# making sure # of rows is correct
gp_pois_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26272 entries, 0 to 26271
Data columns (total 29 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   business_status                  26108 non-null  object 
 1   icon                             26272 non-null  object 
 2   icon_background_color            26272 non-null  object 
 3   icon_mask_base_uri               26272 non-null  object 
 4   name                             26272 non-null  object 
 5   photos                           17188 non-null  object 
 6   place_id                         26272 non-null  object 
 7   price_level                      5248 non-null   float64
 8   rating                           19053 non-null  float64
 9   reference                        26272 non-null  object 
 10  scope                            26272 non-null  object 
 11  types                            26272 non-null  object 
 12  user_ratings_total

In [535]:
gp_pois_merged.head()

,business_status,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,price_level,rating,reference,...,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,permanently_closed,req_cat,result.formatted_address,Borough,Zip,Neighbourhood
0,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Ripe Kitchen & Bar,"[{'height': 2471, 'html_attributions': ['<a hr...",ChIJ9aOdYiyNwokRS49czrNTQo4,2.0,4.3,ChIJ9aOdYiyNwokRS49czrNTQo4,...,-73.840150,True,"V5X6+7F Mount Vernon, NY, USA",87G8V5X6+7F,NaN,restaurant,"151 W Sandford Blvd, Mt Vernon, NY 10550, USA",nan,nan,nan
1,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,New China Garden,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJcVDAYNfywokRPUaKrm0UBpg,1.0,3.9,ChIJcVDAYNfywokRPUaKrm0UBpg,...,-73.854798,True,"V4XW+5M The Bronx, NY, USA",87G8V4XW+5M,NaN,restaurant,"724 Nereid Ave, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
2,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Dunkin',"[{'height': 3000, 'html_attributions': ['<a hr...",ChIJ--58aynzwokRqSvaRDKwX1M,1.0,3.8,ChIJ--58aynzwokRqSvaRDKwX1M,...,-73.850362,True,"V5R2+59 New York, NY, USA",87G8V5R2+59,NaN,restaurant,"980 E 233rd St, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
3,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Subway,"[{'height': 766, 'html_attributions': ['<a hre...",ChIJAy-ibCnzwokRdmkKGd1Yzhw,1.0,3.6,ChIJAy-ibCnzwokRdmkKGd1Yzhw,...,-73.850357,True,"V5R2+58 New York, NY, USA",87G8V5R2+58,NaN,restaurant,"980 E 233rd St, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx
4,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Popeyes Louisiana Kitchen,"[{'height': 1192, 'html_attributions': ['<a hr...",ChIJXVacFSvzwokRa3VpQDuwRDE,1.0,3.8,ChIJXVacFSvzwokRa3VpQDuwRDE,...,-73.844710,True,"V5Q4+QJ New York, NY, USA",87G8V5Q4+QJ,NaN,restaurant,"1201 E 233rd St, The Bronx, NY 10466, USA",Bronx,10466,Northeast Bronx


In [536]:
# making sure all categories are still present
gp_pois_merged['req_cat'].unique()

array(['restaurant', 'school', 'park', 'transit_station', 'supermarket'],
      dtype=object)

In [537]:
gp_pois_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26272 entries, 0 to 26271
Data columns (total 29 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   business_status                  26108 non-null  object 
 1   icon                             26272 non-null  object 
 2   icon_background_color            26272 non-null  object 
 3   icon_mask_base_uri               26272 non-null  object 
 4   name                             26272 non-null  object 
 5   photos                           17188 non-null  object 
 6   place_id                         26272 non-null  object 
 7   price_level                      5248 non-null   float64
 8   rating                           19053 non-null  float64
 9   reference                        26272 non-null  object 
 10  scope                            26272 non-null  object 
 11  types                            26272 non-null  object 
 12  user_ratings_total

In [ ]:
# exporting as csv for final cleaning
gp_pois_merged.to_csv('gp_pois_merged.csv', index=False)